In [ ]:
    async def _async_post_with_retry(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        files: Optional[Dict[str, Any]] = None,
        retries: Optional[int] = 5,
        timeout: Optional[int] = 300,
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous HTTP POST request with retry logic.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            files: Optional files to upload
            retries: Maximum number of retries
            timeout: Request timeout in seconds
            
        Returns:
            Response object or None if all retries failed
        """
        url = self.base_url + path
        remaining_retries = retries
        error = None
        
        while remaining_retries > 0:
            try:
                async with self._semaphore:
                    async with aiohttp.ClientSession() as session:
                        # Handle both JSON payload and form data with files
                        if files:
                            # aiohttp handles multipart form data for files
                            form_data = aiohttp.FormData()
                            
                            # Add any payload fields
                            if payload:
                                for key, value in payload.items():
                                    form_data.add_field(key, str(value))
                            
                            # Add files
                            for key, file_info in files.items():
                                form_data.add_field(
                                    key,
                                    file_info[1],  # File content
                                    filename=file_info[0],  # Filename
                                    content_type=file_info[2]  # Content type
                                )
                                
                            response = await session.post(
                                url,
                                headers=headers,
                                data=form_data,
                                timeout=timeout,
                                ssl=None if not self.verify_ssl else True,
                            )
                        else:
                            # Regular JSON payload
                            response = await session.post(
                                url,
                                headers=headers,
                                json=payload,
                                timeout=timeout,
                                ssl=None if not self.verify_ssl else True,
                            )
                        
                        # Check for HTTP errors
                        if response.status >= 300:
                            detail = f"Error code from API: {response.status} {await response.text()}"
                            logger.error(f"Request failed: {detail}")
                            
                            # For 5xx errors, retry. For 4xx, return the response
                            if response.status < 500:
                                return response
                        else:
                            # Success
                            return response
            
            except Exception as e:
                error = e
                logger.error(
                    f"Error occurred calling API {url}. Details: {str(e)}. "
                    f"Current retry attempt# {retries - remaining_retries + 1} of {retries}."
                )
                
            # If we get here, we need to retry
            remaining_retries -= 1
            if remaining_retries > 0:
                logger.info(f"Sleeping for {self.retry_delay} seconds before next retry")
                await asyncio.sleep(self.retry_delay)
        
        # All retries failed
        logger.error(f"Failed to POST to {url} after {retries} attempts")
        if error:
            logger.error(f"Last error: {str(error)}")
            
        return None